In [88]:
import pandas as pd 
import numpy as np 
import io
from string import punctuation, digits, whitespace 


In [89]:
# Cheap method assuming the given data is already in python:

data = ('Airline Code;DelayTimes;FlightCodes;To_From\n' 
'Air Canada (!);[21, 40];20015.0;WAterLoo_NEWYork\n' 
'<Air France> (12);[];;Montreal_TORONTO\n' 
'(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa\n' 
'12. Air France;[78, 66];;'
'Ottawa_VANcouvER\n' 
'""".\\.Lufthansa.\\.""";[12, 33];20055.0;london_MONTreal\n')

data = io.StringIO(data)
cheat_df = pd.read_csv(data, sep=';')
cheat_df

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015.0,WAterLoo_NEWYork
1,<Air France> (12),[],NaN,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035.0,CALgary_Ottawa
3,12. Air France,"[78, 66]",NaN,Ottawa_VANcouvER
4,""".\.Lufthansa.\.""","[12, 33]",20055.0,london_MONTreal


In [90]:
# More realistic method, wherein data is probably a raw file

with open('data.txt', 'r') as d:
    data = d.readlines()[0].strip('\'').rstrip('\\n')
    data = data.split("\\n")
    df = pd.DataFrame(data)
    df = df[0].str.split(';', expand=True)
    df = df.rename(columns=df.iloc[0]).drop(df.index[0])

df 

,Airline Code,DelayTimes,FlightCodes,To_From
1,Air Canada (!),"[21, 40]",20015.0,WAterLoo_NEWYork
2,<Air France> (12),[],,Montreal_TORONTO
3,(Porter Airways. ),"[60, 22, 87]",20035.0,CALgary_Ottawa
4,12. Air France,"[78, 66]",,Ottawa_VANcouvER
5,""""""".\\.Lufthansa.\\.""""""","[12, 33]",20055.0,london_MONTreal


In [91]:
# Spliting 'To_From' column

df[['To', 'From']] = df['To_From'].str.upper().str.split('_', expand=True)
df = df.drop(['To_From'], axis=1)

# Stripping Airline code of ugly punctuation - neat method without regex

want_gone = punctuation + whitespace + digits
df["Airline Code"] = pd.Series(df["Airline Code"])\
                            .str.strip(want_gone)
df

,Airline Code,DelayTimes,FlightCodes,To,From
1,Air Canada,"[21, 40]",20015.0,WATERLOO,NEWYORK
2,Air France,[],,MONTREAL,TORONTO
3,Porter Airways,"[60, 22, 87]",20035.0,CALGARY,OTTAWA
4,Air France,"[78, 66]",,OTTAWA,VANCOUVER
5,Lufthansa,"[12, 33]",20055.0,LONDON,MONTREAL


In [92]:
# interpolating FlightCodes

df["FlightCodes"] = pd.to_numeric(df["FlightCodes"])\
    .interpolate().astype(np.int64)

# Stripping brackets from DelayTimes

df["DelayTimes"] = pd.Series(df["DelayTimes"])\
                           .str.strip("[]").replace("", np.NaN)

df

,Airline Code,DelayTimes,FlightCodes,To,From
1,Air Canada,"21, 40",20015,WATERLOO,NEWYORK
2,Air France,NaN,20025,MONTREAL,TORONTO
3,Porter Airways,"60, 22, 87",20035,CALGARY,OTTAWA
4,Air France,"78, 66",20045,OTTAWA,VANCOUVER
5,Lufthansa,"12, 33",20055,LONDON,MONTREAL
